In [2]:
import pickle
import time
import pandas as pd
import numpy as np

In [156]:
with open('./PedTables/AllPeds.p', 'rb') as fp:
    AllPeds = pickle.load(fp)

In [157]:
PedIDs = list(AllPeds.keys())
len(PedIDs)

12613

In [158]:
def BothSides(df,axis):       #ped appears on both side of the car axis?
    y = list(df[axis])
    if np.sign(y[0]*y[-1]) == 1:
        both = 0
    else:
        both = 1
    return both

In [159]:
BothSidePeds = {k : v for k, v in AllPeds.items() if BothSides(v,'y') == 1}
BSideIDs = list(BothSidePeds.keys())

In [160]:
len(BSideIDs)

1068

In [161]:
def FrontCross(df):     #ped is in front of car when crossing?
    zero = min((df['y']), key = abs) #closest y to zero (SDC y is 0)
    if np.sign(df[df['y']==zero]['x'].values[0]) == 1:
        front = 1
    else:
        front = 0
    return front


In [162]:
FrontPeds = {k : v for k, v in BothSidePeds.items() if FrontCross(v) == 1}
FCrossIDs = list(FrontPeds.keys())

In [163]:
len(FCrossIDs)

761

In [154]:
CarPassPeds = {k : v for k, v in FrontPeds.items() if BothSides(v,'x') == 1}
CPassIDs = list(CarPassPeds.keys())

In [164]:
len(CPassIDs)

181

In [118]:
FrontPeds[FCrossIDs[1]]

,Timestamp,x,y,z,x_GF,y_GF,pz_GF,x_GF,y_GF,z_GF,...,Speed x,Speed y,accel x,accel y,Veh Count,Ped Count,Bike Count,Weather,Time of Day,Segment ID
0,1.552440e+15,38.892485,-8.550637,0.764913,881.673987,15534.896055,-650.451899,860.594771,15568.674927,-651.441,...,1.056094,1.221171,0.008108,0.009376,31.0,27.0,0.0,sunny,Day,0000_10023947602400723454_1120_000_1140_000
1,1.552440e+15,38.676053,-8.443949,0.844882,881.780164,15535.018829,-650.455803,860.727800,15568.540261,-651.437,...,1.056905,1.222108,0.008108,0.009376,31.0,27.0,0.0,sunny,Day,0000_10023947602400723454_1120_000_1140_000
2,1.552440e+15,38.460068,-8.335287,0.935941,881.886309,15535.141565,-650.459707,860.859179,15568.404793,-651.433,...,1.057716,1.223046,0.008108,0.009376,32.0,27.0,0.0,sunny,Day,0000_10023947602400723454_1120_000_1140_000
3,1.552440e+15,38.246200,-8.225452,0.952301,881.992446,15535.264292,-650.463610,860.992139,15568.270112,-651.433,...,1.058527,1.223984,0.008108,0.009376,32.0,27.0,0.0,sunny,Day,0000_10023947602400723454_1120_000_1140_000
4,1.552440e+15,38.038281,-8.096616,0.922858,882.098618,15535.387060,-650.467515,861.124834,15568.135861,-651.434,...,1.059338,1.224921,0.008108,0.009376,32.0,27.0,0.0,sunny,Day,0000_10023947602400723454_1120_000_1140_000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,1.552440e+15,27.883100,9.138887,0.986134,892.190260,15550.218844,-650.727428,866.036658,15563.537935,-651.469,...,-0.222286,0.244128,0.047694,-0.028795,36.0,35.0,1.0,sunny,Day,0000_10023947602400723454_1120_000_1140_000
195,1.552440e+15,27.848765,9.139995,0.982744,892.166826,15550.243986,-650.729829,866.036751,15563.538036,-651.469,...,-0.217517,0.241249,0.047694,-0.028795,36.0,35.0,1.0,sunny,Day,0000_10023947602400723454_1120_000_1140_000
196,1.552440e+15,27.814404,9.141187,0.982900,892.143399,15550.269121,-650.732229,866.036655,15563.538016,-651.469,...,-0.212747,0.238369,0.047694,-0.028795,36.0,35.0,1.0,sunny,Day,0000_10023947602400723454_1120_000_1140_000
197,1.552440e+15,27.780102,9.142311,0.980385,892.119964,15550.294264,-650.734630,866.036645,15563.538064,-651.469,...,-0.207978,0.235490,0.047694,-0.028795,36.0,35.0,1.0,sunny,Day,0000_10023947602400723454_1120_000_1140_000


In [155]:
FrontPeds[FCrossIDs[1]].columns

Index(['Timestamp', 'x', 'y', 'z', 'x_GF', 'y_GF', 'pz_GF', 'x_GF', 'y_GF',
       'z_GF', 'Width', 'Length', 'Height', 'Heading', 'Speed x', 'Speed y',
       'accel x', 'accel y', 'Veh Count', 'Ped Count', 'Bike Count', 'Weather',
       'Time of Day', 'Segment ID'],
      dtype='object')

In [127]:
CarPassPeds[CPassIDs[1]]

,Timestamp,x,y,z,x_GF,y_GF,pz_GF,x_GF,y_GF,z_GF,...,Speed x,Speed y,accel x,accel y,Veh Count,Ped Count,Bike Count,Weather,Time of Day,Segment ID
0,1.558466e+15,32.091149,9.192908,-0.071304,-6773.253785,8474.464404,-136.080968,-6788.058928,8504.372948,-136.883,...,-1.387369,0.348898,-2.468479e-13,-1.653533e-12,96.0,45.0,0.0,sunny,Day,0010_15379350264706417068_3120_000_3140_000
1,1.558466e+15,31.748948,9.321775,-0.146200,-6773.392650,8474.499326,-136.080968,-6788.023961,8504.167390,-136.883,...,-1.387369,0.348898,-2.468479e-13,-1.653533e-12,98.0,45.0,0.0,sunny,Day,0010_15379350264706417068_3120_000_3140_000
2,1.558466e+15,31.408289,9.449132,-0.202288,-6773.531525,8474.534251,-136.080968,-6787.989484,8503.964395,-136.882,...,-1.387369,0.348898,-2.468479e-13,-1.653533e-12,101.0,45.0,0.0,sunny,Day,0010_15379350264706417068_3120_000_3140_000
3,1.558466e+15,31.071338,9.572939,-0.195865,-6773.670452,8474.569188,-136.080968,-6787.957655,8503.764163,-136.879,...,-1.387369,0.348898,-2.468479e-13,-1.653533e-12,101.0,45.0,0.0,sunny,Day,0010_15379350264706417068_3120_000_3140_000
4,1.558466e+15,30.744349,9.680640,-0.155136,-6773.809376,8474.604125,-136.080968,-6787.928854,8503.569023,-136.874,...,-1.387369,0.348898,-2.468479e-13,-1.653533e-12,103.0,46.0,0.0,sunny,Day,0010_15379350264706417068_3120_000_3140_000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,1.558466e+15,-6.224860,-6.741202,0.971306,-6795.513408,8486.341293,-136.000968,-6786.994834,8482.897556,-136.844,...,-1.206958,0.330030,-3.136782e-01,-4.477473e-01,111.0,34.0,0.0,sunny,Day,0010_15379350264706417068_3120_000_3140_000
195,1.558466e+15,-6.707028,-6.688366,0.956024,-6795.639426,8486.377847,-136.000968,-6786.909850,8482.674301,-136.844,...,-1.238326,0.285255,-3.136782e-01,-4.477473e-01,111.0,34.0,0.0,sunny,Day,0010_15379350264706417068_3120_000_3140_000
196,1.558466e+15,-7.195450,-6.620395,0.950184,-6795.765431,8486.414398,-136.000968,-6786.816779,8482.450031,-136.845,...,-1.269694,0.240480,-3.136782e-01,-4.477473e-01,111.0,34.0,0.0,sunny,Day,0010_15379350264706417068_3120_000_3140_000
197,1.558466e+15,-7.690376,-6.535684,0.944243,-6795.891479,8486.450960,-136.000968,-6786.717345,8482.223887,-136.847,...,-1.301062,0.195706,-3.136782e-01,-4.477473e-01,111.0,33.0,0.0,sunny,Day,0010_15379350264706417068_3120_000_3140_000
